# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [21]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice
import pandas as pd

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [22]:

def dna_to_rna(s):
    converter = {
        # A dictionary as converter
        "A" : "A",
        "C" : "C",
        "G" : "G",
        "T" : "U"
    }
    if not (all([x in converter.keys() for x in s])):
        # Check if all the characters in the string are valid
        raise KeyError("Invalid DNA string.")
    return "".join(list(map(lambda x: converter[x], s)))

if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))
    print("")
    #print(dna_to_rna("B"))

AACGUGAUUUC



### Idea of solution

I create a dictionary 'converter' to map the nucleotides from DNA to RNA. I conducted a validity check to make sure input string only contains 'A', 'C', 'G', and 'T', an KeyError will be raised otherwise. Then I use map() and join() methods to create the converted string.

### Discussion

For valid input and empty string, the conversion is successful. For input string containing invalid characters, an exception will be raised.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [23]:

def get_dict():
    table = {
        "UUU": "F", "UUC": "F", "UUA": "L", "UUG": "L",
        "UCU": "S", "UCC": "S", "UCA": "S", "UCG": "S",
        "UAU": "Y", "UAC": "Y", "UAA": "*", "UAG": "*",
        "UGU": "C", "UGC": "C", "UGA": "*", "UGG": "W",
        "CUU": "L", "CUC": "L", "CUA": "L", "CUG": "L",
        "CCU": "P", "CCC": "P", "CCA": "P", "CCG": "P",
        "CAU": "H", "CAC": "H", "CAA": "Q", "CAG": "Q",
        "CGU": "R", "CGC": "R", "CGA": "R", "CGG": "R",
        "AUU": "I", "AUC": "I", "AUA": "I", "AUG": "M",
        "ACU": "T", "ACC": "T", "ACA": "T", "ACG": "T",
        "AAU": "N", "AAC": "N", "AAA": "K", "AAG": "K",
        "AGU": "S", "AGC": "S", "AGA": "R", "AGG": "R",
        "GUU": "V", "GUC": "V", "GUA": "V", "GUG": "V",
        "GCU": "A", "GCC": "A", "GCA": "A", "GCG": "A",
        "GAU": "D", "GAC": "D", "GAA": "E", "GAG": "E",
        "GGU": "G", "GGC": "G", "GGA": "G", "GGG": "G"
    }
    return table
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L', 'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S', 'UAU': 'Y', 'UAC': 'Y', 'UAA': '*', 'UAG': '*', 'UGU': 'C', 'UGC': 'C', 'UGA': '*', 'UGG': 'W', 'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L', 'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P', 'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M', 'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T', 'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R', 'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V', 'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', 'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}


### Idea of solution

I created the dictionary in get_dict() method manually. The conversion is correspondent with the table in the link. I use '*' to indicate the end.

### Discussion

The dictionary is printed.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [24]:
def get_dict_list():
    codon_to_aa = get_dict()
    aa_to_codons = {}
    for codon, aa in codon_to_aa.items():
        # reverse the k,v relationship in codon_to_aa, so that we get aa_to_codons
        if aa not in aa_to_codons:
            aa_to_codons[aa] = []
        aa_to_codons[aa].append(codon)
    return aa_to_codons
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], '*': ['UAA', 'UAG', 'UGA'], 'C': ['UGU', 'UGC'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'Q': ['CAA', 'CAG'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'I': ['AUU', 'AUC', 'AUA'], 'M': ['AUG'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'E': ['GAA', 'GAG'], 'G': ['GGU', 'GGC', 'GGA', 'GGG']}


### Idea of solution

First I obtained the conversion from codon to amino acid using get_dict() method. Then I made the reversed conversion table using items() method of dict. 

### Discussion

I print the reversed conversion table, the output is a dict with format {amino_acid: [codon1, ...], ...}.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [25]:

def rna_to_prot(s):
    if (len(s) % 3 != 0 and len(s) > 0):
        raise ValueError("Invalid RNA string. Length should be a multiple of 3.")
    codon_to_aa = get_dict()
    aas = []
    for i in range(0, len(s), 3):
        codon = s[i:i+3]
        aa = codon_to_aa[codon]
        aas.append(aa)
        if aa == "*":
            break
    return "".join(aas)

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))
    #print(dna_to_prot("ATGATATCATCGACGATGTAGT"))

MISSTM*


### Idea of solution

First I deal with invalid input (length is not a multiple of 3) string by raising an error. Then I do the conversion and store the result into a list. The return value is constructed using join() method. 

### Discussion

I tested the conversion, and it's successful. Invalid input will raise an ValueError

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [26]:

def get_probabability_dict():
    prob = {
        "UUU":0.4641342698782968, "UUC":0.5358657301217032, "UUA":0.07656764558436285, "UUG":0.1290578537068707,
        "UCU":0.18758584014344437, "UCC":0.21795953165920925, "UCA":0.15051714801827132, "UCG":0.05439771371883908,
        "UAU":0.443338109266921, "UAC":0.5566618907330789, "UAA":0.29701911804823383, "UAG":0.23673791389873997,
        "UGU":0.45615733050366836, "UGC":0.5438426694963316, "UGA":0.46624296805302623, "UGG":1.00,
        "CUU":0.13171591206484023, "CUC":0.1955768259144855, "CUA":0.0713801723134756, "CUG":0.39570159041596514,
        "CCU":0.2867313296570278, "CCC":0.3234703981288551, "CCA":0.2766025276376192, "CCG":0.11319574457649788,
        "CAU":0.4185152128433691, "CAC":0.5814847871566309, "CAA":0.26501675921017337, "CAG":0.7349832407898266,
        "CGU":0.08010752804820104, "CGC":0.18377662978978224, "CGA":0.1088124833207855, "CGG":0.20155434006721587,
        "AUU":0.36107219301206106, "AUC":0.4698662895003286, "AUA":0.16906151748761036, "AUG":1.00,
        "ACU":0.2467688440166039, "ACC":0.35523154074391966, "ACA":0.28418772983891855, "ACG":0.11381188540055791,
        "AAU":0.4703669907468028, "AAC":0.5296330092531971, "AAA":0.43404935110207155, "AAG":0.5659506488979285,
        "AGU":0.14960182300967595, "AGC":0.23993794345056002, "AGA":0.21465774794262568, "AGG":0.21109127083138968,
        "GUU":0.18177011180348712, "GUC":0.23830637956135173, "GUA":0.11657741053350681, "GUG":0.46334609810165434,
        "GCU":0.26592161421413735, "GCC":0.39978112134364696, "GCA":0.2281212631716276, "GCG":0.10617600127058811,
        "GAU":0.4645424191930427, "GAC":0.5354575808069573, "GAA":0.42245266280361615, "GAG":0.5775473371963838,
        "GGU":0.1630865131632947, "GGC":0.33710935809444503, "GGA":0.24992165149690412, "GGG":0.24988247724535617
    }
    return prob
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

I create the dictionary manually.

### Discussion

The output of the dictionary pass the test successfully

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [27]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        aa_to_codons = get_dict_list()
        prob = get_probabability_dict()
        res = []
        for aa in s:
            codons = aa_to_codons[aa]
            probs = [prob[codon] for codon in codons]
            max_index = np.argmax(probs)
            res.append(codons[max_index])
        return "".join(res)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The thought is to use argmax method to obtain the index of the maximal probability, and use this index to retrieve the corresponding codon (as the for ... in ... loop retrieve the element in order).

### Discussion

The output is "CUGACCCCCAUCCAGAACAGAGCC" for "LTPIQNRA", which is correct. The method also pass the test.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [28]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    res = choice(list(dist.keys()), p=list(dist.values()))
    return res

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

C, T, A, G, A, C, T, C, T, T, C, A, A, C, C, G, C, C, A, C, T, C, T, T, T, C, G, C, C


### Idea of solution

I will take advantage of the choice method.

### Discussion

The generated random neucleotide is printed in cell above. The function pass the test.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [29]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        aa_to_codons = get_dict_list()
        prob = get_probabability_dict() # get the probability dictionary
        res = []
        for aa in s:
            codons = aa_to_codons[aa] # get all codons for the amino acid
            probs = [prob[codon] for codon in codons]  
            probs_sum = sum(probs)
            probs = [x/probs_sum for x in probs]  # normalize the probabilities
            codon = random_event(dict(zip(codons, probs))) # sample a random codon based on the probabilities
            res.append(codon)
        return "".join(res)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUUACCCCUAUUCAAAACAGAGCG


### Idea of solution

I took the advantage of the random_event() method to generate the codon according to their probability. The problem is that probability sum up to 1.01 or 0.99 for some amino acid, so I do scaling to let them sum up to 1.

### Discussion

The result is randomly without setting up a seed. The method pass the test. 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [30]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    window = {}
    for i in range(k):
        window[s[i]] = window.get(s[i], 0) + 1
    yield window

    for i in range(k, len(s)):
        window[s[i-k]] -= 1 # remove the first element from the window
        window[s[i]] = window.get(s[i], 0) + 1 # add the new element to the window
        yield window
        
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)

{'T': 1, 'C': 3}
{'T': 0, 'C': 3, 'G': 1}
{'T': 0, 'C': 2, 'G': 1, 'A': 1}
{'T': 0, 'C': 2, 'G': 1, 'A': 1}
{'T': 0, 'C': 1, 'G': 2, 'A': 1}
{'T': 0, 'C': 1, 'G': 2, 'A': 1}
{'T': 0, 'C': 2, 'G': 2, 'A': 0}
{'T': 0, 'C': 2, 'G': 2, 'A': 0}
{'T': 1, 'C': 2, 'G': 1, 'A': 0}
{'T': 2, 'C': 2, 'G': 0, 'A': 0}
{'T': 2, 'C': 1, 'G': 1, 'A': 0}
{'T': 2, 'C': 1, 'G': 1, 'A': 0}
{'T': 1, 'C': 2, 'G': 1, 'A': 0}


### Idea of solution

Nucleotide frequencies in the window is calculated in 2 part. The first part calculate the first window, while the second part calculate the following ones iteratively. 

### Discussion

The output is correct, starting from the first one to the last one. The function pass the test.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [31]:
def context_list(s, k):
    window = {}
    for i in range(len(s) - k):
        context = s[i:i+k]
        next_char = s[i+k]
        if context not in window:
            window[context] = [next_char]
        else:
            window[context].append(next_char)
    window = {k: "".join(v) for k, v in window.items()}
    return window
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

I iterate through all the context according to the window size, and recorded their relative associatives into a list. At the end I convert the list into string, so that each context substring has a corresponding associative string.

### Discussion

The output is correct.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [32]:
def context_probabilities(s, k):
    d = context_list(s, k)
    d = {k: {x: d[k].count(x)/len(d[k]) for x in set(d[k])} for k in d.keys()}
    return d
    
if __name__ == '__main__':
    print(context_probabilities("ATGATATCATCGACGATCTAG", 2))
    print(context_probabilities("ATGATATCATCGACGATCTAG", 0))

{'AT': {'A': 0.2, 'G': 0.2, 'C': 0.6}, 'TG': {'A': 1.0}, 'GA': {'C': 0.3333333333333333, 'T': 0.6666666666666666}, 'TA': {'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.3333333333333333, 'T': 0.3333333333333333, 'G': 0.3333333333333333}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'CT': {'A': 1.0}}
{'': {'A': 0.3333333333333333, 'G': 0.19047619047619047, 'C': 0.19047619047619047, 'T': 0.2857142857142857}}


### Idea of solution

I just need to modify the result returned by context_list(s, k) a little bit, by replacing the values with another dict() that record the proportion of each character.

### Discussion

When k = 2, the result is printed in cell above.
When k = 0, the result represent the zero-order probability of the whole sequence.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [33]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        if seed is not None:
            np.random.seed(seed)
        res = []
        if n == 0:
            return ""
        elif n < self.k:
            # if n is less than k, we can only use the zeroth order model
            for i in range(n):
                res.append(random_event(self.zeroth))
            return res
        for i in range(self.k):
            # generate the first k characters
            res.append(random_event(self.zeroth))
        for i in range(n-self.k):
            # generate the rest of the characters
            context = "".join(res[i:i+self.k])
            next_char = random_event(self.kth[context])
            res.append(next_char)
        return "".join(res)

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

TGTATGATGA


### Idea of solution

In the generate method, first I considered the situation that expected length n is shorter than k, where all characters in result should be generated by zero-order probability. Otherwise the first k character is generated by zeroth, and following ones are generated using kth.

### Discussion

The result is "TGTATGATGA" when seed is set to 0, the function pass the test

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [34]:
def context_probabilities_initial_1(s, k):
    d = context_list(s, k)
    for k in d.keys():
        d[k] = d[k] + "ACTG" # add frequency_count=1 for each nucleotide
    d = {k: {x: d[k].count(x)/len(d[k]) for x in set(d[k])} for k in d.keys()} # calculate the probabilities
    return d

def context_pseudo_probabilities(s, k):
    d = context_probabilities_initial_1(s, k)
    if k == 0:
        for k in d.keys():
            for kk in d[k].keys():
                d[k][kk] = round(d[k][kk], 2) # round the probabilities to 2 decimal places
        return d
    all_possible_context = ["".join(x) for x in product("ACGT", repeat=k)]
    for context in all_possible_context:
        if context not in d.keys():
            d[context] = {x:0.25 for x in "ACGT"} # if the context is not present, add it with equal probabilities
        else:
            for x in "ACGT":
                if x not in d[context].keys():
                    d[context][x] = 0.0
    return d
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'G': 0.24, 'C': 0.16, 'T': 0.28}
AT: {'A': 0.2222222222222222, 'G': 0.3333333333333333, 'T': 0.1111111111111111, 'C': 0.3333333333333333}
TG: {'A': 0.3333333333333333, 'G': 0.16666666666666666, 'C': 0.16666666666666666, 'T': 0.3333333333333333}
GA: {'A': 0.14285714285714285, 'G': 0.14285714285714285, 'C': 0.2857142857142857, 'T': 0.42857142857142855}
TA: {'A': 0.16666666666666666, 'G': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.16666666666666666, 'G': 0.3333333333333333}
CA: {'A': 0.2, 'G': 0.2, 'C': 0.2, 'T': 0.4}
CG: {'A': 0.5, 'G': 0.16666666666666666, 'T': 0.16666666666666666, 'C': 0.16666666666666666}
AC: {'A': 0.2, 'G': 0.4, 'T': 0.2, 'C': 0.2}
GT: {'A': 0.4, 'G': 0.2, 'T': 0.2, 'C': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 

### Idea of solution

I modify the context_probabilities() method a little bit, by appending "ACGT" to each value which is equal to setting the initial frequency count to 1. The Pseudo part includes adding "ACGT" for context that doesn't appear in the training set.

### Discussion

The output is correct and the function pass the test

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [35]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        if self.k == 0:
            return np.prod([self.zeroth[x] for x in s])
        base = []
        if len(s) <= self.k:
            for i in range(len(s)):
                base.append(s[i])
            return np.prod([self.zeroth[x] for x in base])
        base = [s[i] for i in range(self.k)]
        context = [s[i:i+self.k] for i in range(len(s)-self.k)]
        return np.prod([self.kth[context[i]][s[i+self.k]] for i in range(len(context))]) * np.prod([self.zeroth[x] for x in base])

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {'A'} is {mc.probability('A')}")
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence A is 0.32
Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340018e-10


### Idea of solution

First, I consider the situation when k=0, where the result is computed using zeroth. When k>0, I first consider the situation when len(s) <= k, where no context is included, so I still use the zeroth to compute the probability. When len(s) > k, the probability comes from 2 part: base part representing the first k elements, and context part representing the following ones.

### Discussion

The result is printed, and the function pass the test

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [36]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        if self.k == 0:
            return np.prod([self.zeroth[x] for x in s])
        base = []
        if len(s) <= self.k:
            for i in range(len(s)):
                base.append(s[i])
            return np.log2(np.prod([self.zeroth[x] for x in base]))
        base = [s[i] for i in range(self.k)]
        context = [s[i:i+self.k] for i in range(len(s)-self.k)]
        return np.log2(np.prod([self.kth[context[i]][s[i+self.k]] for i in range(len(context))]) * np.prod([self.zeroth[x] for x in base]))
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538307


### Idea of solution

I just use np.log2() method to transform the output in problem 14.

### Discussion

The result is correct.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [37]:
def better_context_probabilities(s, k):
    return context_pseudo_probabilities(s, k)

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

AT: {'A': 0.2222222222222222, 'G': 0.3333333333333333, 'T': 0.1111111111111111, 'C': 0.3333333333333333}
TG: {'A': 0.3333333333333333, 'G': 0.16666666666666666, 'C': 0.16666666666666666, 'T': 0.3333333333333333}
GA: {'A': 0.14285714285714285, 'G': 0.14285714285714285, 'C': 0.2857142857142857, 'T': 0.42857142857142855}
TA: {'A': 0.16666666666666666, 'G': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.16666666666666666, 'G': 0.3333333333333333}
CA: {'A': 0.2, 'G': 0.2, 'C': 0.2, 'T': 0.4}
CG: {'A': 0.5, 'G': 0.16666666666666666, 'T': 0.16666666666666666, 'C': 0.16666666666666666}
AC: {'A': 0.2, 'G': 0.4, 'T': 0.2, 'C': 0.2}
GT: {'A': 0.4, 'G': 0.2, 'T': 0.2, 'C': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T':

### Idea of solution

It seems my solution in exercise 13 already satisfy the requirement in exercise 16, this is because in the context_probabilities_initial_1() method, I directly calculate the frequency for each k:v pairs and return the frequency dictionary.

### Discussion

my solution in exercise 13 pass the test for exercise 16

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [38]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k

    def generate(self, n, seed=None):
        if seed is not None:
            np.random.seed(seed)

        if self.k == 0:
            return "".join([random_event(context_pseudo_probabilities(self.s, 0)[""]) for i in range(n)])
        
        if n <= self.k:
            return self.s[:n]
        
        state = self.s[:self.k]
        result = state

        for i in range(n-self.k):
            context = result[-self.k:]
            next_char = self._sample_next(context)
            result += next_char

        return result

    def _sample_next(self, context):
        possible_next = [i for i in range(len(self.s) - self.k) if self.s[i:i+self.k] == context]
        if not possible_next:
            return "".join([random_event(context_pseudo_probabilities(self.s, 0)[""]) for i in range(1)])
        next_index = np.random.choice(possible_next)
        return self.s[next_index + self.k]
        
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    seed = 7
    mc = SimpleMarkovChain(s, k)
    for n in range(40):
        print(f"generate {mc.generate(n, seed)}")


generate 
generate A
generate AT
generate ATG
generate ATGT
generate ATGTA
generate ATGTAT
generate ATGTATC
generate ATGTATCG
generate ATGTATCGA
generate ATGTATCGAT
generate ATGTATCGATA
generate ATGTATCGATAT
generate ATGTATCGATATG
generate ATGTATCGATATGT
generate ATGTATCGATATGTA
generate ATGTATCGATATGTAT
generate ATGTATCGATATGTATC
generate ATGTATCGATATGTATCA
generate ATGTATCGATATGTATCAT
generate ATGTATCGATATGTATCATG
generate ATGTATCGATATGTATCATGT
generate ATGTATCGATATGTATCATGTA
generate ATGTATCGATATGTATCATGTAT
generate ATGTATCGATATGTATCATGTATG
generate ATGTATCGATATGTATCATGTATGA
generate ATGTATCGATATGTATCATGTATGAT
generate ATGTATCGATATGTATCATGTATGATG
generate ATGTATCGATATGTATCATGTATGATGT
generate ATGTATCGATATGTATCATGTATGATGTA
generate ATGTATCGATATGTATCATGTATGATGTAG
generate ATGTATCGATATGTATCATGTATGATGTAGA
generate ATGTATCGATATGTATCATGTATGATGTAGAT
generate ATGTATCGATATGTATCATGTATGATGTAGATG
generate ATGTATCGATATGTATCATGTATGATGTAGATGT
generate ATGTATCGATATGTATCATGTATGATGTAGATGTA
generate A

### Idea of solution

First I consider the situation when k==0, where I just use random_event() and context_pseudo_probabilities() methods to generate the result. When k>0, the result is based on the context: If matched context(s) exist, the next_char is generated based on the probability of the context. If no matched context found, the nex_char is generated on top of the zeroth probability of the character in the string s.

### Discussion

I print the output in cell above using a loop, with n ranging from 0 to 40. The function pass the test

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [39]:
def kmer_index(s, k):
    index = {}
    for i in range(len(s) - k + 1):
        kmer = s[i:i+k]
        if kmer not in index:
            index[kmer] = [i]
        else:
            index[kmer].append(i)
    return index

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}


### Idea of solution

This is a simple task, just record the first index of each context to a dictionary.

### Discussion

The result is printed, for example the index for 'AT' is [0, 3, 5, 8, 15]. The function pass the test

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [40]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    prob = better_context_probabilities(rna, 3)
    return prob
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    divergence = 0.0
    for event, prob in p.items():
        if q.get(event, 0) == 0 and prob != 0:
            raise ZeroDivisionError
        elif prob != 0:
            divergence += prob * np.log2(prob/q[event])
    return divergence

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    pass
    
    # Maybe check that converting back to protein results in the same sequence
    pass
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities("<rna sequence>") # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = 0.0


ZeroDivisionError: 

### Idea of solution

In codon_probabilities() method, I just use the better_context_probabilities(rna, 3) method to calculate the probabilities.

In kullback_leibler(p, q), ZeroDivisionError will be raised if a probability in q is 0 and p is not. Otherwise I will just skip the calculation (when probability in p is 0) or update the divergence (when probability in p is not 0)

### Discussion

The function pass the test

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [ ]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.162, -0.227, +0.388, +0.151
-0.370, +0.057, +0.198, +0.037


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [ ]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[ 0.3   0.    0.7   0.  ]
 [ 0.    0.4   0.    0.6 ]
 [ 0.35  0.    0.65  0.  ]
 [ 0.    0.2   0.    0.8 ]]
Stationary distributions:
[[ 0.02421173 -0.29026212  0.1389851   0.44623574]
 [ 0.24261234  0.09071543  0.35909807 -0.47662849]]
Using [0.24, 0.09, 0.36, -0.48] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.60666836
KL divergence of stationary distribution prefix of length    10 is 0.71775085
KL divergence of stationary distribution prefix of length   100 is 0.38115273
KL divergence of stationary distribution prefix of length  1000 is 0.36116144
KL divergence of stationary distribution prefix of length 10000 is 0.22844669


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [ ]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[ 0.3   0.1   0.5   0.1 ]
 [ 0.2   0.3   0.15  0.35]
 [ 0.25  0.15  0.2   0.4 ]
 [ 0.35  0.2   0.4   0.05]]
Equilibrium distribution:
[ 0.05549613 -0.0618828  -0.3045492   0.11913485]

Using [-0.3337754   0.45068768 -0.10300145 -0.20088342] as initial distribution:
kl-divergence   empirical distribution
0.25023050057   [ 0.2243623   0.15184879 -0.36068593  0.0442362 ]
0.85018673369   [-0.27420015 -0.1344076  -0.08342149  0.10517491]
0.69016371310   [-0.11457606  0.27115112  0.16359241 -0.1280264 ]
0.14692236781   [ 0.07706555  0.23608696  0.01647088 -0.08926859]
0.98884289212   [-0.06341222  0.45084425  0.3957673  -0.40553139]

Using [-0.14182502  0.21310283  0.21065106  0.45942999] as initial distribution:
kl-divergence   empirical distribution
0.09700875402   [ 0.35101425 -0.18876535  0.25445247 -0.20776396]
0.51559517427   [ 0.31812104 -0.03010157 -0.2113726  -0.49508434]
0.92023091580   [ 0.40268876 -0.22995087 -0.42421415 -0.1464154 ]
0.32096833450  

### Idea of solution

fill in

### Discussion
fill in